In [141]:
%%bash
pip install --upgrade pip
pip install smart_open minecart
pip install textract-trp

In [142]:
import time 
import re
import os
import trp
import boto3
import minecart
import json
import logging 

import numpy as np
import pandas as pd

from smart_open import open
from sagemaker.session import Session

In [143]:
# initiate s3 bucket and corresponding data folder
bucket = "ran-s3-systemic-risk"
data_folder ="Input/X-17A-5-Subsets/"

# script to perform OCR (using Textract) for X-17A-5 subsets
out_folder = 'Output/X-17A-5-BS/'

# Amazon Textract client and Sagemaker session
textract = boto3.client('textract')
s3 = boto3.client('s3')
session = Session()

In [144]:
paths = np.array(session.list_s3_files(bucket, out_folder))[1:]

**We begin by first stripping away NaN terms in the first column and then mapping all the NaN terms to an empty string**

In [145]:
for csv in paths:
    fileName = csv.split('/')[-1]

    # work on combining columns that are issued seperately
    s3.download_file(bucket, csv, 'temp.pdf')
    df = pd.read_csv('temp.pdf')

    # first begin by filtering out the NaN rows present in the first column
    filterDF = df[np.isin(df[df.columns[0]], df[df.columns[0]].dropna())]
    filterDF = filterDF.fillna('')

    # writing data frame to .csv file
    filterDF.to_csv(fileName, index=False)

    # save contents to AWS S3 bucket
    with open(fileName, 'rb') as data:
        s3.put_object(Bucket=bucket, Key=out_folder + fileName, Body=data)

    # remove local file after it has been created
    os.remove(fileName)

    # remove local file after it has been created
    os.remove('temp.pdf')
    
print('All .csv files are cleaned of NaN terms')

All .csv files are cleaned of NaN terms


## Table column merging
**For tables with three columns we merge the last two columns into a once unique column**

In [149]:
def singular_merge(df:pd.DataFrame) -> pd.DataFrame:
    """
    Function passes a special dataframe, and reduces its dimensions accordingly
    - Example releases include but are note limited to 1224385-2016, 72267-2003
    ----
    e.g.
    
    Converts a wide dataframe, balance sheet into a smaller rectangular form
                  0                                                 1                 2
            ====================================================================================
        0   Assets                      
        1   Cash and cash equivalents                       | $ 606,278      |     
        2   Cash and securities segregated pursuant         | 273,083        | 
        3   Collateralized short-term financing agreements: | NaN            | $ 1,345
    
    
    Rectangular form of the the dataframe ->
                   0                                                 1          
            =====================================================================
        0   Assets                      
        1   Cash and cash equivalents                       | $ 606,278        
        2   Cash and securities segregated pursuant         | 273,083        
        3   Collateralized short-term financing agreements: | $ 1,345            
    """
    cleanDF = pd.DataFrame()
    
    df = df.fillna('')    # fill all NaN values with empty string
    
    # create first column of new dataframe that corresponds with first column in prior data
    cleanDF['0'] = df[df.columns[0]]

    # we assume that the second and third columns are filled with figures
    cleanDF['1'] = df[df.columns[1]] + df[df.columns[2]]
    
    return cleanDF

In [150]:
for csv in paths:
    fileName = csv.split('/')[-1]
    
    # work on combining columns that are issued seperately
    s3.download_file(bucket, csv, 'temp.pdf')
    df = pd.read_csv('temp.pdf')
    
    # if columns greater than 2, we have a weird data table
    if df.columns.size > 2:
        
        # two events could occur at this point (either total splits, or year splits)
        arr = df[df.columns[2]].values
        
        # check the scope of the second column 
        n = arr.size
        k = arr.tolist().count(np.nan)
        
        # if more than half the arr size is np.nan we assume this is a "fake column"
        # we merge these columns since there are many blank rows, otherwise we assume year split 
        if k >= int(n/2):
            tempDF = singular_merge(df)
        else:
            tempDF = df[df.columns[:2]]

        # writing data frame to .csv file
        tempDF.to_csv(fileName, index=False)

        # save contents to AWS S3 bucket
        with open(fileName, 'rb') as data:
            s3.put_object(Bucket=bucket, Key=out_folder + fileName, Body=data)
        
        print('We merged {}'.format(fileName))
        # remove local file after it has been created
        os.remove(fileName)

    # remove local file after it has been created
    os.remove('temp.pdf')

We merged 1224385-2016.csv


## Table Row Split
**Since many of the existing tables run the risk of overlapping rows we work to split these rows to appropriate values**

In [153]:
def row_split(df:pd.DataFrame) -> pd.DataFrame:
    """
    Function designed to split conjoined rows from Balance sheet dataframes
    - Example releases include but are note limited to 42352-2015, 58056-2009, 58056-2013
    
    NOTE: Our objective isn't to achieve a perfect split, but rather create labels easy enough for our predictive 
    model to identify and accurately predict. This is not a perfect method and we make assumptions as to the data 
    """
    
    def find_splits(val) -> bool:
        """
        Compute a boolean measure to assess whether a row is conjoined or not 
        """
        try:
            # split the data figures for each balance sheet figure
            x = val.split(' ')
            
            # remove the $ sign if present in the list (this helps avoid false pasitives) 
            try:
                x.remove('$')
            except ValueError:
                # if no $ found we just ignore
                pass
            
            # if length of read list exceeds 1 then we know there exists a multi-row bunch
            if len(x) > 1:
                return True
            else: return False
        except AttributeError: return False
    
    # select all the rows that match our description, where a space exists 
    selections = df[df[df.columns[1]].apply(lambda x: find_splits(x))]
    idxs = selections.index

    # initialize the reporting dataframe
    temp_df = df

    for i in idxs:

        # slice dataframe according to the idx selection (we search for all periods were a break occurs)
        top = temp_df.loc[:i-1]
        bottom = temp_df.loc[i+1:]

        # divide the identified term from the selection piece
        # e.g. "$ 9,112,943 13,151,663" -> ["$", "9,112,943", "13,151,663"] 
        values = temp_df[temp_df.columns[1]].loc[i].split(' ')

        # remove the $ sign if present in the list, otherwise pass 
        try: values.remove('$')
        except ValueError: pass
        
        lineName = temp_df[temp_df.columns[0]].loc[i]
        split = int(len(lineName) * .66)   # index where to cut the string (we assign a 66% cut-off)

        # forming dataframe from dictionary, we then re-map columns and index values (these are new rows)
        # zip restricts bounds to left most split (so always two rows are returned)
        # e.g. dict(zip(['A', 'B'], [1, 2, 3, 4])) -> {'A': 1, 'B': 2}
        mid = pd.DataFrame.from_dict(dict(zip([lineName[:split], lineName[split:]], values)), 
                                     orient='index').reset_index()
        mid.columns = ['0', '1']
        mid.index = [0, 0]

        # reassign the value of df2 to update across each iteration
        temp_df = pd.concat([top, mid, bottom])
        
    return temp_df

In [ ]:
for csv in paths:
    fileName = csv.split('/')[-1]
    
    # work on combining columns that are issued seperately
    s3.download_file(bucket, csv, 'temp.pdf')
    df = pd.read_csv('temp.pdf')
    
    tempDF = row_split(df)
    
    # if difference is found then 
    if tempDf.shape != df.shape:
        print("Fixed the rows for {}".format(fileName))
        
        # writing data frame to .csv file
        tempDF.to_csv(fileName, index=False)

        # save contents to AWS S3 bucket
        with open(fileName, 'rb') as data:
            s3.put_object(Bucket=bucket, Key=out_folder + fileName, Body=data)

        # remove local file after it has been created
        os.remove(fileName)

    # remove local file after it has been created
    os.remove('temp.pdf')
    
print('\nWe fixed all conjoined tables in sample')

## Table Totals Removal
**Some tables have rows that record the intermediate totals for a particular line item category**

In [161]:
# issues with unpacking total values from tables 42352-2005
s3.download_file(bucket, 'Output/X-17A-5-BS/42352-2005.csv', 'temp.pdf')

df = pd.read_csv('temp.pdf')

In [162]:
df

,0,1
0,Cash and cash equivalents,"$ 665,532"
1,Cash and securities segregated in compliance w...,NaN
2,and other regulations,"33,665,205"
3,"Receivables from brokers, dealers and clearing...","8,974,752"
4,Receivables from customers and counterparties,"12,342,912"
5,Securities borrowed,"193,784,018"
6,Securities purchased under agreements to resell,"30,376,416"
7,"Financial instruments owned, at fair value","50,257,864"
8,Financial instruments owned and pledged as col...,"15,194,916"
9,"Total financial instruments owned, at fair value","65,452,780"


In [122]:
# # check for the word "cash" in a string at the begining, ignoring case sensitivity (asset check)
# df[df.columns[0]].str.contains('^Total', regex=True, flags=re.IGNORECASE)

In [133]:
a = df[df.columns[1]].values.tolist()

In [134]:
a.count(np.nan)

4